## TODO:

- use framework of cabinet class
- focus on first item
- find the handle obj file

### environment
using conda env partfield (python 3.19)

In [1]:

import json
import os

path = '/home/wujoseph/github/articulate-partfield/data/partnet-mobility-v0/dataset'

folders = [name for name in os.listdir(path)
           if os.path.isdir(os.path.join(path, name))]

# list of all id in PartNet-Mobility


id_dict = {}
for item in folders:
    # find the class of this 
    with open(f'/home/wujoseph/github/MARS/data/partnet-mobility-v0/dataset/{item}/meta.json', 'r', encoding='utf-8') as f:
        # 使用 json.load() 將檔案內容解析成 Python 字典
        data = json.load(f)

    _class = data['model_cat']

    if _class in id_dict:
        id_dict[_class].append(item)
    else:
        id_dict[_class] = [item]

for cls in id_dict.keys():
    id_dict[cls].sort()
    print(cls, len(id_dict[cls]))

Window 58
Table 101
Clock 31
Camera 37
Laptop 55
StorageFurniture 346
Fan 81
Dishwasher 48
CoffeeMachine 54
Suitcase 24
Faucet 84
Toilet 69
TrashCan 70
Refrigerator 44
Eyeglasses 65
Knife 44
Switch 70
Display 37
Lamp 45
Door 36
Toaster 25
Bottle 57
Stapler 23
Cart 61
Printer 29
Scissors 47
KitchenPot 25
Microwave 16
Pliers 25
Globe 61
Box 28
Dispenser 57
Oven 30
Kettle 29
Keyboard 37
Safe 30
Chair 81
USB 51
Bucket 36
Pen 48
Mouse 14
Remote 49
WashingMachine 17
Lighter 28
Phone 18
FoldingChair 26


In [2]:
print(id_dict['StorageFurniture'])

['35059', '38516', '40147', '40417', '40453', '41003', '41004', '41083', '41085', '41086', '41452', '41510', '41529', '44781', '44817', '44826', '44853', '44962', '45001', '45007', '45087', '45091', '45092', '45130', '45132', '45134', '45135', '45146', '45159', '45162', '45164', '45166', '45168', '45173', '45176', '45177', '45178', '45189', '45194', '45203', '45212', '45213', '45219', '45235', '45238', '45243', '45244', '45247', '45248', '45249', '45261', '45262', '45267', '45271', '45290', '45297', '45305', '45323', '45332', '45354', '45372', '45374', '45378', '45384', '45385', '45387', '45397', '45403', '45413', '45415', '45419', '45420', '45423', '45427', '45443', '45444', '45448', '45463', '45503', '45504', '45505', '45516', '45523', '45524', '45526', '45573', '45575', '45594', '45600', '45606', '45612', '45620', '45621', '45622', '45623', '45632', '45633', '45636', '45638', '45642', '45645', '45661', '45662', '45667', '45670', '45671', '45676', '45677', '45687', '45689', '45690', 

In [3]:
# Create list of full paths for StorageFurniture folders
obj_path_li = [os.path.join(path, folder_id) for folder_id in id_dict['StorageFurniture']]

print(f"Number of StorageFurniture paths: {len(obj_path_li)}")
print("First few paths:")
for i, obj_path in enumerate(obj_path_li[:5]):
    print(f"{i+1}: {obj_path}")

Number of StorageFurniture paths: 346
First few paths:
1: /home/wujoseph/github/articulate-partfield/data/partnet-mobility-v0/dataset/35059
2: /home/wujoseph/github/articulate-partfield/data/partnet-mobility-v0/dataset/38516
3: /home/wujoseph/github/articulate-partfield/data/partnet-mobility-v0/dataset/40147
4: /home/wujoseph/github/articulate-partfield/data/partnet-mobility-v0/dataset/40417
5: /home/wujoseph/github/articulate-partfield/data/partnet-mobility-v0/dataset/40453


In [33]:
def find_keypart_recursive(data, keypart_li:list, keypart_found=None):
    """
    Recursively search for objects with text containing 'Handle' or 'handle'
    and extract the numeric IDs from their 'objs' field
    """
    if keypart_found is None:
        keypart_found = []
    
    # If data is a list, iterate through each item
    if isinstance(data, list):
        for item in data:
            find_keypart_recursive(item, keypart_li, keypart_found)
    
    # If data is a dictionary, check for handle text and objs
    elif isinstance(data, dict):
        # Check if this object has text field containing 'handle' (case insensitive)
        if 'text' in data and isinstance(data['text'], str):
            for keypart in keypart_li:
                if keypart in data['text'].lower():
                    # Extract numeric IDs from objs field
                    if 'objs' in data and isinstance(data['objs'], list):
                        for obj in data['objs']:
                            if isinstance(obj, str) and obj.startswith('original-'):
                                # Extract the numeric part after 'original-'
                                numeric_id = obj.split('-')[-1]
                                try:
                                    keypart_found.append(int(numeric_id))
                                except ValueError:
                                    print(f"Warning: Could not convert '{numeric_id}' to integer from '{obj}'")
        
        # Recursively search in all values of the dictionary
        for key, value in data.items():
            find_keypart_recursive(value, keypart_li, keypart_found)
    
    return keypart_found



In [27]:
def find_handles_recursive(data, handles_found=None):
    """
    Recursively search for objects with text containing 'Handle' or 'handle'
    and extract the numeric IDs from their 'objs' field
    """
    if handles_found is None:
        handles_found = []
    
    # If data is a list, iterate through each item
    if isinstance(data, list):
        for item in data:
            find_handles_recursive(item, handles_found)
    
    # If data is a dictionary, check for handle text and objs
    elif isinstance(data, dict):
        # Check if this object has text field containing 'handle' (case insensitive)
        if 'text' in data and isinstance(data['text'], str):
            if 'handle' in data['text'].lower():
                # Extract numeric IDs from objs field
                if 'objs' in data and isinstance(data['objs'], list):
                    for obj in data['objs']:
                        if isinstance(obj, str) and obj.startswith('original-'):
                            # Extract the numeric part after 'original-'
                            numeric_id = obj.split('-')[-1]
                            try:
                                handles_found.append(int(numeric_id))
                            except ValueError:
                                print(f"Warning: Could not convert '{numeric_id}' to integer from '{obj}'")
        
        # Recursively search in all values of the dictionary
        for key, value in data.items():
            find_handles_recursive(value, handles_found)
    
    return handles_found



In [35]:
# for testing, only process the first object
count = 0
zero_count = 0
keypart_li = ['handle']

for obj_path in obj_path_li:

    with open(os.path.join(obj_path, 'result.json'), 'r', encoding='utf-8') as f:
        data = json.load(f)
        handle_ids = find_keypart_recursive(data, keypart_li)


        #print(obj_path)
        #print(f"Found handle IDs: {handle_ids}")
        #print(f"Number of handles found: {len(handle_ids)}")
        if(len(handle_ids) == 0):
            zero_count += 1

    #if(count > 10):
    #    break
    count += 1
    # break # only process the first object

print(f"{zero_count} / {count}")

28 / 346
